# CSVS to Hepfile

Imagine we have a database like structure with multiple csvs all connected by a common ID. See the household exmaple here: https://hepfile.readthedocs.io/en/latest/fundamentals.html#a-toy-example 

In [1]:
import os, glob
import pandas as pd
import numpy as np
import awkward as ak
import hepfile as hf

In [2]:
def csv_to_awkward(csvpaths:list[str], common_key:str, group_names:list=None) -> ak.Record:
    
    if group_names is None:
        group_names = [os.path.split(file) for file in files]

    for_ak = {}
    for f, group_name in zip(files, group_names):

        csv = pd.read_csv(f)

        groups = csv.groupby(common_key)
        split_groups = []
        for item in groups.groups:
            split_groups.append(groups.get_group(item))

        subdict = {}
        for grouping in split_groups:
            for colname in grouping.columns:
                if colname in subdict.keys():
                    subdict[colname].append(list(grouping[colname].values))
                else:
                    subdict[colname] = [list(grouping[colname].values)]

        for key in subdict.keys():
            subdict[key] = ak.Array(subdict[key])

        for_ak[group_name] = subdict
        print(subdict)
        print()
        
    return ak.Record(for_ak)

In [3]:
datapath = os.path.join('/', 'home', 'nfranz', 'research', 'hepfile', 'docs', 'example_nb', '*.csv')
files = glob.glob(datapath)
common_key = 'Household ID'
group_names = ['Residences', 'People', 'Vehicles']
awk = csv_to_awkward(files, common_key, group_names=group_names)

{'Household ID': <Array [[0], [1], [2], [3]] type='4 * var * int64'>, 'House/apartment/condo': <Array [['House'], ['Apartment'], ..., ['House']] type='4 * var * string'>, '# of bedrooms': <Array [[4], [2], [2], [6]] type='4 * var * int64'>, '# of bathrooms': <Array [[2.5], [2], [1], [4.5]] type='4 * var * float64'>, 'Square footage': <Array [[1500], [1200], [1000], [4500]] type='4 * var * int64'>, 'Year built': <Array [[1955], [2002], [2014], [1998]] type='4 * var * int64'>, 'Estimate': <Array [[250000], [1400], [325000], [500000]] type='4 * var * int64'>}

{'Household ID': <Array [[0, 0, 0, 0], [...], ..., [3, 3, 3, 3, 3, 3, 3]] type='4 * var * int64'>, 'First name': <Array [['blah', 'blah', 'blah', 'blah'], ..., [...]] type='4 * var * string'>, 'Last name': <Array [['blah', 'blah', 'blah', 'blah'], ..., [...]] type='4 * var * string'>, 'Gender ID': <Array [['M', 'F', 'NB', 'F'], ..., ['M', ..., 'F']] type='4 * var * string'>, 'Age': <Array [[54, 52, 18, 14], ..., [54, 52, ..., 11, 65

In [7]:
hf.awkward_tools.awkward_to_hepfile(awk, 'testing_csv_translation.h5')

Adding group Residences
Adding a counter for Residences as nResidences
Adding dataset Household ID to the dictionary under group Residences.
----------------------------------------------------
Slashes / are not allowed in dataset names
Replacing / with - in dataset name House/apartment/condo
The new name will be House-apartment-condo
----------------------------------------------------
Adding dataset House-apartment-condo to the dictionary under group Residences.
Adding dataset # of bedrooms to the dictionary under group Residences.
Adding dataset # of bathrooms to the dictionary under group Residences.
Adding dataset Square footage to the dictionary under group Residences.
Adding dataset Year built to the dictionary under group Residences.
Adding dataset Estimate to the dictionary under group Residences.
Adding group People
Adding a counter for People as nPeople
Adding dataset Household ID to the dictionary under group People.
Adding dataset First name to the dictionary under group P

{'_GROUPS_': {'_SINGLETONS_GROUP_': ['COUNTER'],
  'Residences': ['nResidences',
   'Household ID',
   'House-apartment-condo',
   '# of bedrooms',
   '# of bathrooms',
   'Square footage',
   'Year built',
   'Estimate'],
  'People': ['nPeople',
   'Household ID',
   'First name',
   'Last name',
   'Gender ID',
   'Age',
   'Height',
   'Yearly income',
   'Highest degree-grade'],
  'Vehicles': ['nVehicles',
   'Household ID',
   'Type of vehicle',
   '# of riders',
   'Gas-electric-human powered',
   'Year',
   'Cost']},
 '_MAP_DATASETS_TO_COUNTERS_': {'_SINGLETONS_GROUP_': '_SINGLETONS_GROUP_/COUNTER',
  'Residences': 'Residences/nResidences',
  'Residences/Household ID': 'Residences/nResidences',
  'Residences/House-apartment-condo': 'Residences/nResidences',
  'Residences/# of bedrooms': 'Residences/nResidences',
  'Residences/# of bathrooms': 'Residences/nResidences',
  'Residences/Square footage': 'Residences/nResidences',
  'Residences/Year built': 'Residences/nResidences',
  

In [10]:
data, _ = hf.load('testing_csv_translation.h5', return_awkward=True)

Building the indices...

Built the indices!
Data is read in and input file is closed.
['People' 'Residences' 'Vehicles'] ['People', 'People/Age', 'People/First name', 'People/Gender ID', 'People/Height', 'People/Highest degree-grade', 'People/Household ID', 'People/Last name', 'People/Yearly income', 'People/nPeople', 'Residences', 'Residences/# of bathrooms', 'Residences/# of bedrooms', 'Residences/Estimate', 'Residences/House-apartment-condo', 'Residences/Household ID', 'Residences/Square footage', 'Residences/Year built', 'Residences/nResidences', 'Vehicles', 'Vehicles/# of riders', 'Vehicles/Cost', 'Vehicles/Gas-electric-human powered', 'Vehicles/Household ID', 'Vehicles/Type of vehicle', 'Vehicles/Year', 'Vehicles/nVehicles', '_SINGLETONS_GROUP_', '_SINGLETONS_GROUP_/COUNTER']


In [11]:
data

<Record {People: [{...}, ...], ...} type='{People: 4 * {Age: var * int64, "...'>